In [ ]:
!conda install -y -c conda-forge skorch
!conda install -y pytorch torchvision torchaudio cpuonly -c pytorch
!pip install nb_black

In [1]:
from pathlib import Path

import joblib
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from skorch import NeuralNetClassifier, classifier
from skorch.callbacks import EarlyStopping
from skorch.helper import predefined_split
from torch.nn.modules.loss import L1Loss
from torch.optim import SGD, Adam, RMSprop

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
transformations = torchvision.transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307), ((0.3081))),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.GaussianBlur(kernel_size=3),
    ]
)

training_data = torchvision.datasets.MNIST(root=Path("nn_mnist", "data"), train=True, download=True, transform=transformations)
test_data = torchvision.datasets.MNIST(root=Path("nn_mnist", "data"), train=False, download=True, transform=transforms.ToTensor())

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=60000, shuffle=True, num_workers=0)

# Take batch and label from train dataloader
train_dataloader_batch = next(iter(train_dataloader))
X_train = train_dataloader_batch[0].numpy()
y_train = train_dataloader_batch[1].numpy()

test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=10000, shuffle=False, num_workers=0)
test_dataloader_batch = next(iter(test_dataloader))
X_test = test_dataloader_batch[0].numpy()
y_test = test_dataloader_batch[1].numpy()

<IPython.core.display.Javascript object>

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, 300)
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28 * 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = NeuralNetClassifier(MLP, callbacks=[EarlyStopping()], criterion=nn.CrossEntropyLoss, max_epochs=50)

parameters = [
    {
        "lr": [0.1, 0.001, 0.001],
        "optimizer": [SGD],
        "optimizer__momentum": [0.3, 0.6, 0.9],
    },
    {
        "lr": [0.1, 0.001, 0.001],
        "optimizer": [Adam],
    },
]

gs = GridSearchCV(
    net,
    parameters,
    refit=True,
    cv=5,
    return_train_score=True,
    verbose=3,
    scoring="accuracy",
    n_jobs=-1,
)


gs.fit(X_train, y_train)

joblib.dump(gs, Path("nn_mnist", "results", "NN_gs.pkl"))
gs.best_estimator_.save_params(f_params=Path("nn_mnist", "results", "NN_weights.pkl"))

# Grid Search Parameter Table
grid_table = pd.concat(
    [
        pd.DataFrame(gs.cv_results_["params"]),
        pd.DataFrame(gs.cv_results_["mean_test_score"], columns=["Accuracy"]),
    ],
    axis=1,
)

grid_table.to_csv(Path("nn_mnist", "results", "NN_grid_table.csv"))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6825       0.8815        0.3632  1.4399
      2        0.2816       0.9078        0.2860  1.0634
      3        0.2014       0.9194        0.2512  1.1279
      4        0.1547       0.9278        0.2275  1.1674
      5        0.1230       0.9293        0.2307  1.1517
      6        0.0989       0.9339        0.2242  1.1857
      7        0.0800       0.9328        0.2358  1.1461
      8        0.0650       0.9353        0.2422  1.3187
      9        0.0541       0.9360        0.2512  1.1430
     10        0.0455       0.9319        0.2705  1.2005
Stopping since valid_loss has not improved in the last 5 epochs.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  60 | elapsed:   55.3s remaining:  7.0min
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.8min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  49 out of  60 | elapsed:  2.7min remaining:   36.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.8min finished


<IPython.core.display.Javascript object>

In [1]:
gs = joblib.load(Path("nn_mnist", "results", "NN_gs.pkl"))

Net = gs.best_estimator_.initialize()
Net.load_params(Path("nn_mnist", "results", "NN_weights.pkl"))
Net.check_is_fitted

y_pred = Net.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

NameError: name 'joblib' is not defined

In [5]:
SVM = SVC(
    gamma='scale',
)

params = [
    {
        "kernel": ["rbf", "linear"], 
        "C": [3, 6, 9],
    }
]


SVM_gs = GridSearchCV(
    SVM,
    param_grid=params,
    refit=True,
    cv=2,
    return_train_score=True,
    verbose=3,
    scoring="balanced_accuracy",
    n_jobs=-1,
)


X_train_flat = X_train.squeeze().reshape((60000, 28 * 28))

SVM_gs.fit(X_train_flat, y_train)

joblib.dump(SVM_gs, Path("nn_mnist", "results", "SVM_gs.pkl"))

grid_table = pd.concat(
    [
        pd.DataFrame(SVM_gs.cv_results_["params"]),
        pd.DataFrame(SVM_gs.cv_results_["mean_test_score"], columns=["Accuracy"]),
    ],
    axis=1,
)

grid_table.to_csv(Path("nn_mnist", "results", "SVM_grid_table.csv"))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed: 17.2min remaining: 86.2min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed: 40.6min remaining: 29.0min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 77.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 77.6min finished


<IPython.core.display.Javascript object>

In [14]:
gs = joblib.load(Path("nn_mnist", "results", "SVM_gs.pkl"))

X_test_flat = X_test.squeeze().reshape((10000, 28 * 28))

y_pred = SVM_gs.best_estimator_.predict(X_test_flat)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

<IPython.core.display.Javascript object>